In [1]:
import requests
import json
# r = requests.get("https://api.opendota.com/api/players/91886888/matches?region=2&limit=5")
# with open('matches_sri_5.json', 'w') as data_file:
#     json.dump(r.json(),data_file)

In [2]:
# import bigjson

# with open('E:\yasp-dump.json\yasp-dump.json', 'rb') as f:
#     j = bigjson.load(f)
#     element = j[4]
#     print(element)

# import ijson

# with open('E:\yasp-dump.json\yasp-dump.json', 'rb') as input_file:
#         # load json iteratively
#         parser = ijson.parse(input_file)
#         chat = ijson.items(input_file, 'features.item.chat.item')
#         for item in chat:
#             print('Chat: {}'.format(item))
# #         for prefix, event, value in parser:
# #             print('prefix={}, event={}, value={}'.format(prefix, event, value))

In [3]:
wins = {}
losses = {}
both = {}

# with open('matches_sri_5.json', 'r') as data_file:
with open('games.json', 'r', encoding="utf-8") as data_file:
    data = json.load(data_file)
    
    for element in data:
#         print(element)
        element['match_id'] = str(element['match_id']).replace('[', '').replace(']', '')
        r = requests.get("https://api.opendota.com/api/matches/" + str(element['match_id']))
        r = r.json()

        r.pop('all_word_counts', None)
        r.pop('match_id', None)
        r.pop('barracks_status_dire', None)
        r.pop('barracks_status_radiant', None)
#         r.pop('chat', None)
        r.pop('cluster', None)
        r.pop('cosmetics', None)
        r.pop('dire_score', None)
        r.pop('dire_team_id', None)
        r.pop('draft_timings', None)
        r.pop('duration', None)
        r.pop('engine', None)
#         r.pop('first_blood_time', None)
#         r.pop('game_mode', None)
        r.pop('human_players', None)
        r.pop('leagueid', None)
        r.pop('lobby_type', None)
        r.pop('match_seq_num', None)
        r.pop('negative_votes', None)
        r.pop('objectives', None)
        r.pop('picks_bans', None)
        r.pop('positive_votes', None)
        r.pop('radiant_gold_adv', None)
        r.pop('radiant_score', None)
        r.pop('radiant_team_id', None)
#         r.pop('radiant_win', None)
        r.pop('radiant_xp_adv', None)
        r.pop('skill', None)
        r.pop('start_time', None)
        r.pop('teamfights', None)
        r.pop('tower_status_dire', None)
        r.pop('tower_status_radiant', None)
        r.pop('version', None)
        r.pop('replay_salt', None)
        r.pop('series_id', None)
        r.pop('series_type', None)
        r.pop('players', None)
        r.pop('patch', None)
        r.pop('region', None)
        r.pop('throw', None)
        r.pop('loss', None)
        r.pop('win', None)
        r.pop('my_word_counts', None)
        r.pop('comeback', None)
        r.pop('stomp', None)
        r.pop('replay_url', None)
     
        print(r)
        pop_chat = []
        for index, item in enumerate(r['chat']):
            if item['slot'] == 10:
                pop_chat.append(index)

        print("pop index" + str(pop_chat))
        for item in reversed(pop_chat):
            print("Popped" + str(r['chat'].pop(int(item))))
        
        with open('chat_wheel.json', 'r',encoding='utf8') as data_file:
            chat_wheel = json.load(data_file)
            
        for idx, item in enumerate(r['chat']):
            if item['type'] == 'chatwheel':
#                 print(item['key'])
                item['type'] = 'chat'
                item['key'] = chat_wheel[item['key']].get('message')
        
        both[element['match_id']] = r
        print("Added to both.")
#         print(r)
        
        if r['radiant_win'] == True:
            wins[element['match_id']] = r
            print("Added to wins.")
        else:
            losses[element['match_id']] = r
            print("Added to losses.")
        with open('wins.json', 'w') as outfile:
            json.dump(wins, outfile)
        with open('losses.json', 'w') as outfile:
            json.dump(losses, outfile)
        with open('both.json', 'w') as outfile:
            json.dump(both, outfile)
            
# print("Wins:")
# print(wins)
# print("Losses:")
# print(losses)
# print("Both:")
# print(both)


{'chat': [{'time': -21, 'type': 'chatwheel', 'key': '71', 'slot': 3, 'player_slot': 3}, {'time': 149, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 175, 'type': 'chatwheel', 'key': '7', 'slot': 0, 'player_slot': 0}, {'time': 226, 'type': 'chatwheel', 'key': '71', 'slot': 0, 'player_slot': 0}, {'time': 263, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 381, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 388, 'type': 'chatwheel', 'key': '71', 'slot': 3, 'player_slot': 3}, {'time': 405, 'type': 'chatwheel', 'key': '71', 'slot': 4, 'player_slot': 4}, {'time': 478, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 494, 'type': 'chatwheel', 'key': '71', 'slot': 7, 'player_slot': 130}, {'time': 494, 'type': 'chatwheel', 'key': '71', 'slot': 7, 'player_slot': 130}, {'time': 560, 'type': 'chatwheel', 'key': '71', 'slot': 3, 'player_slot': 3}, {'time': 570, 'type': 'chatwheel', 'key': '2', 'slo

{'chat': [{'time': -225, 'type': 'chat', 'unit': '>.<', 'key': '回来对面就秒了', 'slot': 10}, {'time': -222, 'type': 'chat', 'unit': '>.<', 'key': '\ue048', 'slot': 10}, {'time': -77, 'type': 'chatwheel', 'key': '78', 'slot': 2, 'player_slot': 2}, {'time': -56, 'type': 'chatwheel', 'key': '4002', 'slot': 5, 'player_slot': 128}, {'time': 281, 'type': 'chatwheel', 'key': '71', 'slot': 9, 'player_slot': 132}, {'time': 282, 'type': 'chatwheel', 'key': '71', 'slot': 9, 'player_slot': 132}, {'time': 319, 'type': 'chatwheel', 'key': '71', 'slot': 9, 'player_slot': 132}, {'time': 319, 'type': 'chatwheel', 'key': '71', 'slot': 9, 'player_slot': 132}, {'time': 411, 'type': 'chatwheel', 'key': '71', 'slot': 9, 'player_slot': 132}, {'time': 503, 'type': 'chatwheel', 'key': '71', 'slot': 6, 'player_slot': 129}, {'time': 503, 'type': 'chatwheel', 'key': '71', 'slot': 6, 'player_slot': 129}, {'time': 665, 'type': 'chatwheel', 'key': '84001', 'slot': 3, 'player_slot': 3}, {'time': 671, 'type': 'chatwheel', '

{'chat': [{'time': -60, 'type': 'chatwheel', 'key': '62', 'slot': 4, 'player_slot': 4}, {'time': -17, 'type': 'chatwheel', 'key': '14003', 'slot': 8, 'player_slot': 131}, {'time': 2, 'type': 'chatwheel', 'key': '7', 'slot': 5, 'player_slot': 128}, {'time': 11, 'type': 'chatwheel', 'key': '14003', 'slot': 8, 'player_slot': 131}, {'time': 11, 'type': 'chatwheel', 'key': '14001', 'slot': 8, 'player_slot': 131}, {'time': 14, 'type': 'chatwheel', 'key': '14002', 'slot': 8, 'player_slot': 131}, {'time': 153, 'type': 'chatwheel', 'key': '1', 'slot': 1, 'player_slot': 1}, {'time': 250, 'type': 'chatwheel', 'key': '5', 'slot': 5, 'player_slot': 128}, {'time': 291, 'type': 'chatwheel', 'key': '58', 'slot': 1, 'player_slot': 1}, {'time': 293, 'type': 'chatwheel', 'key': '2', 'slot': 1, 'player_slot': 1}, {'time': 296, 'type': 'chatwheel', 'key': '2', 'slot': 1, 'player_slot': 1}, {'time': 312, 'type': 'chatwheel', 'key': '71', 'slot': 4, 'player_slot': 4}, {'time': 312, 'type': 'chatwheel', 'key'

Added to both.
Added to losses.
{'chat': [{'time': 4, 'type': 'chatwheel', 'key': '76', 'slot': 3, 'player_slot': 3}, {'time': 88, 'type': 'chatwheel', 'key': '1', 'slot': 3, 'player_slot': 3}, {'time': 94, 'type': 'chatwheel', 'key': '1', 'slot': 3, 'player_slot': 3}, {'time': 141, 'type': 'chatwheel', 'key': '71', 'slot': 3, 'player_slot': 3}, {'time': 142, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 253, 'type': 'chatwheel', 'key': '76', 'slot': 3, 'player_slot': 3}, {'time': 259, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 267, 'type': 'chatwheel', 'key': '71', 'slot': 6, 'player_slot': 129}, {'time': 271, 'type': 'chatwheel', 'key': '71', 'slot': 2, 'player_slot': 2}, {'time': 280, 'type': 'chatwheel', 'key': '2', 'slot': 2, 'player_slot': 2}, {'time': 388, 'type': 'chatwheel', 'key': '71', 'slot': 6, 'player_slot': 129}, {'time': 396, 'type': 'chatwheel', 'key': '71', 'slot': 6, 'player_slot': 129}, {'time': 406, 'type': '

Added to both.
Added to wins.
{'chat': [{'time': -571, 'type': 'chatwheel', 'key': '7', 'slot': 6, 'player_slot': 129}, {'time': -241, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': 'Da da ya', 'slot': 6, 'player_slot': 129}, {'time': -226, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': '� � � � � � � �', 'slot': 6, 'player_slot': 129}, {'time': -226, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': '� � � � � � � �', 'slot': 6, 'player_slot': 129}, {'time': -225, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': '� � � � � � � �', 'slot': 6, 'player_slot': 129}, {'time': -224, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': '� � � � � � � �', 'slot': 6, 'player_slot': 129}, {'time': -224, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': '� � � � � � � �', 'slot': 6, 'player_slot': 129}, {'time': -223, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': '� � � � � � � �', 'slot': 6, 'player_slot': 129}, {'time': -216, 'type': 'chat', 'unit': 'SAD INSIDE', 'key': 'sosat', 'slot': 6, 'player_slot': 129}, {'time': 

...

{'chat': [{'time': -15, 'type': 'chat', 'unit': 'aRtQ', 'key': ')))', 'slot': 6, 'player_slot': 129}, {'time': 12, 'type': 'chat', 'unit': 'jorka.ka(usher)', 'key': 'как ты с крюка взял', 'slot': 3, 'player_slot': 3}, {'time': 14, 'type': 'chat', 'unit': 'jorka.ka(usher)', 'key': 'руну', 'slot': 3, 'player_slot': 3}, {'time': 20, 'type': 'chat', 'unit': 'jorka.ka(usher)', 'key': 'сучара', 'slot': 3, 'player_slot': 3}, {'time': 244, 'type': 'chatwheel', 'key': '72', 'slot': 6, 'player_slot': 129}, {'time': 245, 'type': 'chatwheel', 'key': '72', 'slot': 6, 'player_slot': 129}, {'time': 358, 'type': 'chatwheel', 'key': '74', 'slot': 1, 'player_slot': 1}, {'time': 360, 'type': 'chatwheel', 'key': '7', 'slot': 8, 'player_slot': 131}, {'time': 399, 'type': 'chatwheel', 'key': '3', 'slot': 2, 'player_slot': 2}, {'time': 424, 'type': 'chatwheel', 'key': '74', 'slot': 1, 'player_slot': 1}, {'time': 459, 'type': 'chatwheel', 'key': '71', 'slot': 2, 'player_slot': 2}, {'time': 461, 'type': 'chatw

{'chat': [{'time': -80, 'type': 'chatwheel', 'key': '62', 'slot': 8, 'player_slot': 131}, {'time': -80, 'type': 'chatwheel', 'key': '62', 'slot': 8, 'player_slot': 131}, {'time': -60, 'type': 'chatwheel', 'key': '7', 'slot': 5, 'player_slot': 128}, {'time': -11, 'type': 'chatwheel', 'key': '7', 'slot': 8, 'player_slot': 131}, {'time': -11, 'type': 'chatwheel', 'key': '7', 'slot': 8, 'player_slot': 131}, {'time': 246, 'type': 'chatwheel', 'key': '0', 'slot': 8, 'player_slot': 131}, {'time': 249, 'type': 'chatwheel', 'key': '71', 'slot': 2, 'player_slot': 2}, {'time': 249, 'type': 'chatwheel', 'key': '71', 'slot': 2, 'player_slot': 2}, {'time': 264, 'type': 'chatwheel', 'key': '71', 'slot': 5, 'player_slot': 128}, {'time': 323, 'type': 'chatwheel', 'key': '10', 'slot': 1, 'player_slot': 1}, {'time': 336, 'type': 'chatwheel', 'key': '71', 'slot': 3, 'player_slot': 3}, {'time': 336, 'type': 'chatwheel', 'key': '71', 'slot': 3, 'player_slot': 3}, {'time': 363, 'type': 'chatwheel', 'key': '2

{'chat': [{'time': -52, 'type': 'chatwheel', 'key': '7', 'slot': 2, 'player_slot': 2}, {'time': -52, 'type': 'chatwheel', 'key': '7', 'slot': 2, 'player_slot': 2}, {'time': -50, 'type': 'chatwheel', 'key': '7', 'slot': 3, 'player_slot': 3}, {'time': -50, 'type': 'chatwheel', 'key': '7', 'slot': 3, 'player_slot': 3}, {'time': 97, 'type': 'chat', 'unit': 'o0o(>,<)o0o', 'key': 'SORY', 'slot': 5, 'player_slot': 128}, {'time': 97, 'type': 'chat', 'unit': 'Kurama', 'key': 'gak usah lagi normal ', 'slot': 8, 'player_slot': 131}, {'time': 97, 'type': 'chat', 'unit': 'Tsubasa_gt', 'key': 'wet my friend pls', 'slot': 4, 'player_slot': 4}, {'time': 97, 'type': 'chat', 'unit': 'BadBoY', 'key': 'zzz', 'slot': 6, 'player_slot': 129}, {'time': 97, 'type': 'chat', 'unit': 'o0o(>,<)o0o', 'key': 'WET?', 'slot': 5, 'player_slot': 128}, {'time': 97, 'type': 'chat', 'unit': 'ma 吃白', 'key': 'u wet ur fren>', 'slot': 7, 'player_slot': 130}, {'time': 97, 'type': 'chat', 'unit': 'Kurama', 'key': 'wet anjer', '

{'chat': [{'time': 3, 'type': 'chatwheel', 'key': '61', 'slot': 6, 'player_slot': 129}, {'time': 4, 'type': 'chatwheel', 'key': '2', 'slot': 6, 'player_slot': 129}, {'time': 6, 'type': 'chatwheel', 'key': '61', 'slot': 6, 'player_slot': 129}, {'time': 184, 'type': 'chatwheel', 'key': '2', 'slot': 1, 'player_slot': 1}, {'time': 185, 'type': 'chatwheel', 'key': '2', 'slot': 1, 'player_slot': 1}, {'time': 195, 'type': 'chat', 'unit': 'AMARENDRAJ7B', 'key': 'ese meepo ', 'slot': 5, 'player_slot': 128}, {'time': 214, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 347, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 347, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 377, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 378, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 536, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 609, 'type': 'chatw

{'chat': [{'time': 235, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 238, 'type': 'chatwheel', 'key': '71', 'slot': 2, 'player_slot': 2}, {'time': 291, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 362, 'type': 'chatwheel', 'key': '71', 'slot': 2, 'player_slot': 2}, {'time': 369, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 507, 'type': 'chatwheel', 'key': '71', 'slot': 1, 'player_slot': 1}, {'time': 957, 'type': 'chatwheel', 'key': '70', 'slot': 1, 'player_slot': 1}, {'time': 957, 'type': 'chatwheel', 'key': '65', 'slot': 1, 'player_slot': 1}, {'time': 957, 'type': 'chatwheel', 'key': '69', 'slot': 1, 'player_slot': 1}, {'time': 1198, 'type': 'chat', 'unit': '死', 'key': '认识吗 不认识等多久阿', 'slot': 4, 'player_slot': 4}, {'time': 1198, 'type': 'chat', 'unit': '死', 'key': '半天没脸进来了', 'slot': 4, 'player_slot': 4}], 'first_blood_time': 205, 'game_mode': 18, 'radiant_win': True}
pop index[]
Added to both.
Added to 

In [4]:
wins

{'5116468346': {'chat': [{'time': -27,
    'type': 'chat',
    'key': '%s1 has returned!',
    'slot': 0,
    'player_slot': 0},
   {'time': -27,
    'type': 'chat',
    'key': '%s1 has returned!',
    'slot': 0,
    'player_slot': 0},
   {'time': -20,
    'type': 'chat',
    'key': 'Well played!',
    'slot': 0,
    'player_slot': 0},
   {'time': -14,
    'type': 'chat',
    'key': '%s1 has returned!',
    'slot': 0,
    'player_slot': 0},
   {'time': -13,
    'type': 'chat',
    'key': '%s1 has returned!',
    'slot': 0,
    'player_slot': 0},
   {'time': 9,
    'type': 'chat',
    'key': 'Well played!',
    'slot': 5,
    'player_slot': 128},
   {'time': 66,
    'type': 'chat',
    'key': 'Thanks!',
    'slot': 5,
    'player_slot': 128},
   {'time': 66,
    'type': 'chat',
    'key': 'Thanks!',
    'slot': 5,
    'player_slot': 128},
   {'time': 115,
    'type': 'chat',
    'key': 'Well played!',
    'slot': 0,
    'player_slot': 0},
   {'time': 159,
    'type': 'chat',
    'key': 

In [5]:
import copy
win_chats={}
lose_chats={}
for game in wins:
    game_copy = copy.deepcopy(wins[game])
    game_copy['chat']=[line for line in wins[game]['chat'] if not (line['slot']>4)]
    #print(game_copy)
    win_chats[game]=game_copy
    
    game_copy = copy.deepcopy(wins[game])
    game_copy['chat']=[line for line in wins[game]['chat'] if not (line['slot']<=4)]
    #print(game_copy)
    lose_chats[game]=game_copy

for game in losses:
    game_copy = copy.deepcopy(losses[game])
    game_copy['chat']=[line for line in losses[game]['chat'] if not (line['slot']>4)]
    lose_chats[game]=game_copy
    
    game_copy = copy.deepcopy(losses[game])
    game_copy['chat']=[line for line in losses[game]['chat'] if not (line['slot']<=4)]
    #print(game_copy)
    win_chats[game]=game_copy

print(win_chats) 
print(len(win_chats))
print(lose_chats) 
len(lose_chats)


{'5116468346': {'chat': [{'time': -27, 'type': 'chat', 'key': '%s1 has returned!', 'slot': 0, 'player_slot': 0}, {'time': -27, 'type': 'chat', 'key': '%s1 has returned!', 'slot': 0, 'player_slot': 0}, {'time': -20, 'type': 'chat', 'key': 'Well played!', 'slot': 0, 'player_slot': 0}, {'time': -14, 'type': 'chat', 'key': '%s1 has returned!', 'slot': 0, 'player_slot': 0}, {'time': -13, 'type': 'chat', 'key': '%s1 has returned!', 'slot': 0, 'player_slot': 0}, {'time': 115, 'type': 'chat', 'key': 'Well played!', 'slot': 0, 'player_slot': 0}, {'time': 159, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 1, 'player_slot': 1}, {'time': 161, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 1, 'player_slot': 1}, {'time': 266, 'type': 'chat', 'key': 'Well played!', 'slot': 0, 'player_slot': 0}, {'time': 407, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 4, 'player_slot': 4}, {'time': 408, 'type': 'chat', 'key': '%s1 is missing!', 'slot': 4, 'player_slot': 4}, {'time': 440, 'type': 'chat', 

33

In [6]:
wordDict = {}
for game in both.keys():
    for i in both[game]['chat']:
        for word in i['key'].split():
            if word in wordDict:
                wordDict[word] = wordDict[word]+1
            else:
                wordDict[word]=1

In [7]:
for i in both.keys():
    print(i)

5116156624
5116468346
5116470655
5116453043
5116465564
5116466617
5116450613
5116452869
5116463143
5116447876
5116421259
5116453569
5116451709
5116453222
5116453867
5116467779
5116460493
5116455885
5116457175
5116445767
5116444480
5116411910
5116432404
5116450761
5116451909
5116443654
5116439354
5116439508
5116425789
5116439421
5116439146
5116444884
5116440889


In [8]:
wordDict

{'%s1': 342,
 'is': 354,
 'missing!': 337,
 'Well': 246,
 'played!': 246,
 'Get': 202,
 'Back!': 200,
 'All': 2,
 'enemy': 3,
 'heroes': 2,
 'lol': 13,
 'Good': 114,
 'game,': 86,
 'well': 86,
 'played': 86,
 'Wp': 3,
 'GGWP': 1,
 'gg': 22,
 'has': 9,
 'returned!': 7,
 'Thanks!': 27,
 'We': 37,
 'need': 35,
 'wards.': 35,
 'u': 21,
 'can': 4,
 'end': 17,
 'бездарь': 1,
 'на': 6,
 'вокере': 1,
 'иди': 2,
 'закончи': 1,
 'У': 1,
 'меня': 1,
 'хуйня': 1,
 'шмотки': 1,
 'ломает': 1,
 'Follow': 1,
 'me': 15,
 'game': 28,
 'пупу': 1,
 'gege': 2,
 'Report': 1,
 'drow': 1,
 '...': 3,
 '+': 1,
 'shya': 1,
 'magnus': 1,
 'nazreshsyaa': 1,
 'derma': 1,
 'Affirmative': 8,
 'хахах': 1,
 'ты': 8,
 'еблан?': 1,
 'ту': 1,
 'игру': 2,
 'слил': 2,
 'что': 3,
 'по': 1,
 'говну': 1,
 '?)': 1,
 'теперь': 1,
 'эту?': 1,
 'хахаха': 1,
 'D': 8,
 'ward': 2,
 'postavil': 1,
 'ff': 1,
 'УХАХАХХА': 1,
 'САСАТЬ': 1,
 'морф,': 1,
 'же': 1,
 'хотел': 1,
 'заставить': 1,
 'жрать': 1,
 'говно': 1,
 'гирыча': 1,
 'наше

In [9]:
import operator
wordDictCounts = sorted(wordDict.items(), key=operator.itemgetter(1))

In [10]:
wordDictCounts[::-1]

[('is', 354),
 ('%s1', 342),
 ('missing!', 337),
 ('played!', 246),
 ('Well', 246),
 ('Get', 202),
 ('Back!', 200),
 ('Good', 114),
 ('Missing', 88),
 ('played', 86),
 ('well', 86),
 ('game,', 86),
 ('�', 84),
 ('ha', 83),
 ('mid!', 44),
 ('We', 37),
 ('wards.', 35),
 ('need', 35),
 ('Careful!', 32),
 ('Missing!', 30),
 ('Help!', 29),
 ('hm', 29),
 ('game', 28),
 ('Thanks!', 27),
 ('now', 25),
 ('Push', 25),
 ('top!', 23),
 ('gg', 22),
 ('Nice', 21),
 ('bottom!', 21),
 ('u', 21),
 ('?', 19),
 ('end', 17),
 ('ha!', 16),
 ('Ha', 16),
 ('Go!', 16),
 ('just', 16),
 ('hard', 15),
 ('Game', 15),
 ('Okay.', 15),
 ('me', 15),
 ('noob', 14),
 ('haha', 14),
 ('lol', 13),
 ('wet', 12),
 ('Whoops', 12),
 ('that', 12),
 ('to', 12),
 ('my', 11),
 ('no', 11),
 ('bad', 10),
 ('My', 10),
 ('me?', 10),
 ('Was', 10),
 ('Oops.', 10),
 ('sky', 9),
 ('Sorry', 9),
 ('ha.', 9),
 ('Hm', 9),
 ('happened', 9),
 ('That', 9),
 ('now!', 9),
 ('Stun', 9),
 ('pls', 9),
 ('we', 9),
 ('has', 9),
 ('AW', 8),
 ('I', 8),


In [11]:
words = list(wordDict.keys())

In [12]:
words

['%s1',
 'is',
 'missing!',
 'Well',
 'played!',
 'Get',
 'Back!',
 'All',
 'enemy',
 'heroes',
 'lol',
 'Good',
 'game,',
 'well',
 'played',
 'Wp',
 'GGWP',
 'gg',
 'has',
 'returned!',
 'Thanks!',
 'We',
 'need',
 'wards.',
 'u',
 'can',
 'end',
 'бездарь',
 'на',
 'вокере',
 'иди',
 'закончи',
 'У',
 'меня',
 'хуйня',
 'шмотки',
 'ломает',
 'Follow',
 'me',
 'game',
 'пупу',
 'gege',
 'Report',
 'drow',
 '...',
 '+',
 'shya',
 'magnus',
 'nazreshsyaa',
 'derma',
 'Affirmative',
 'хахах',
 'ты',
 'еблан?',
 'ту',
 'игру',
 'слил',
 'что',
 'по',
 'говну',
 '?)',
 'теперь',
 'эту?',
 'хахаха',
 'D',
 'ward',
 'postavil',
 'ff',
 'УХАХАХХА',
 'САСАТЬ',
 'морф,',
 'же',
 'хотел',
 'заставить',
 'жрать',
 'говно',
 'гирыча',
 'нашего',
 'пуште',
 "I'm",
 'retreating',
 'So',
 'much',
 'blood.',
 'Hee',
 'hee',
 'he',
 'aha',
 'ha...',
 "What're",
 'we',
 'laughing',
 'about?',
 'Okay.',
 'Kiss',
 'Crybaby',
 'dont',
 'meepo',
 'pls',
 'es',
 'second',
 'report',
 'here',
 'ya',
 'dog',


In [13]:
words.insert(0,'Radiant Wins')

In [14]:
import numpy as np
AllWords = np.zeros((len(both)*2,len(wordDict)+1), dtype=int)
game_index = 0
for game in win_chats.keys():
    AllWords[game_index][0]=1
    for line in win_chats[game]['chat']:
        for word in line['key'].split():
            AllWords[game_index][words.index(word)]=AllWords[game_index][words.index(word)]+1
    game_index+=1
for game in lose_chats.keys():
    for line in lose_chats[game]['chat']:
        for word in line['key'].split():
            AllWords[game_index][words.index(word)]=AllWords[game_index][words.index(word)]+1
    game_index+=1

In [15]:
AllWords

array([[ 1, 15, 11, ...,  0,  0,  0],
       [ 1,  4,  4, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  4,  4, ...,  0,  0,  0],
       [ 0,  4,  4, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [16]:
import pandas as pd 
df = pd.DataFrame(AllWords, columns = words) 

In [17]:
df

,Radiant Wins,%s1,is,missing!,Well,played!,Get,Back!,All,enemy,...,ggwp,g,kunkka,:*,aja,udh,rubik,0-9,dc,DL
0,1,15,11,11,20,20,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4,4,4,2,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,12,12,12,4,4,6,6,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,13,13,13,0,0,3,3,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,7,8,7,11,11,4,4,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,3,12,3,4,4,8,8,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,3,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [20]:
y_test

array([1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [23]:
y_pred = classifier.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2 3]
 [2 7]]
              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.70      0.78      0.74         9

    accuracy                           0.64        14
   macro avg       0.60      0.59      0.59        14
weighted avg       0.63      0.64      0.63        14

